### Initial setup

In [1]:
import cv2
import os
import json
from utils import *

images_dir = './samples'
input = 'input.json'
output = 'output.json'

### Check image type

In [2]:
with open(input, 'r') as json_file:
    data = json.load(json_file)

image_paths = data['image_files']

for index, image_path in enumerate(image_paths):
    full_path = os.path.join(images_dir, image_path)
    image = cv2.imread(full_path)
    if image is not None:
        image = cv2.resize(image, (0, 0), fx=0.15, fy=0.15)
        complexity, num_keypoints = get_image_complexity(image)
        print(f'Image {index} ({image_path}): Complexity: {complexity} ({num_keypoints} features)')

Image 0 (0_4UMA_original_60479_1609884606260.jpg): Complexity: 1 (82 features)
Image 1 (0_hWqG_original_3460_1609895876431.jpg): Complexity: 1 (55 features)
Image 2 (0_hXAf_original_3460_1609895891398.jpg): Complexity: 1 (39 features)
Image 3 (0_MnDu_original_15254_1609945083260.jpg): Complexity: 1 (20 features)
Image 4 (0_neOX_original_3460_1609895884436.jpg): Complexity: 1 (8 features)
Image 5 (0_rGIB_original_15254_1609945086846.jpg): Complexity: 1 (16 features)
Image 6 (1_utXT_original-test_1609981179609.jpg): Complexity: 1 (30 features)
Image 7 (2_7mUR_original-test_1609981034838.jpg): Complexity: 1 (63 features)
Image 8 (2_GPI9_original-test_1609981038128.jpg): Complexity: 1 (28 features)
Image 9 (2_lXUX_original-test_1609981023743.jpg): Complexity: 1 (17 features)
Image 10 (2_vUVe_original-test_1609981148397.jpg): Complexity: 1 (31 features)
Image 11 (IMG_20201127_001711.jpg): Complexity: 1 (206 features)
Image 12 (IMG_20201127_001800.jpg): Complexity: 2 (307 features)
Image 13 

### GrabCut testing

In [6]:
# ----------------------------------------------

with open(input, 'r') as json_file:
    data = json.load(json_file)

image_paths = data['image_files']
image_index = 1

image = cv2.imread(os.path.join(images_dir, image_paths[image_index]))
image = cv2.resize(image, (0, 0), fx = 0.15, fy = 0.15)

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

complexity, _ = get_image_complexity(image)

# ----------------------------------------------

blur_kernel = (7, 7)
bb_size = 10
iterations = 3
grabcut_mask = grab_cut(image, blur_kernel, bb_size, iterations)
grabcut = cv2.bitwise_and(image, image, mask=grabcut_mask)

# ----------------------------------------------

cv2.imshow('Image', image)
cv2.imshow('Grabcut', grabcut)
cv2.waitKey(0)
cv2.destroyAllWindows()

# ----------------------------------------------

### Background median testing

In [3]:
# ----------------------------------------------

with open(input, 'r') as json_file:
    data = json.load(json_file)

image_paths = data['image_files']
image_index = 44

original_image = cv2.imread(os.path.join(images_dir, image_paths[image_index]))
original_image = cv2.resize(original_image, (0, 0), fx = 0.15, fy = 0.15)
image = original_image.copy()

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

complexity, _ = get_image_complexity(gray)

# ----------------------------------------------

def process_image():
    complexity, _ = get_image_complexity(gray)
    blur_kernel = (5, 5)
    if complexity == 2:
        blur_kernel = (5, 5)
    elif complexity == 3:
        blur_kernel = (11, 11)
    bg_median_mask = background_median(hsv, hue_tolerance, saturation_tolerance, value_tolerance, blur_kernel)
    bg_median = cv2.bitwise_and(image, image, mask=bg_median_mask)
    contours, _ = cv2.findContours(bg_median_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = filter_contours(contours, original_image)
    image_copy = image.copy()
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(image_copy, (x, y), (x + w, y + h), (255, 255, 0), 2)
    combined_image = np.hstack((bg_median, image_copy))
    cv2.imshow(window_name, combined_image)

def update_hue_tolerance(*args):
    global hue_tolerance
    hue_tolerance = cv2.getTrackbarPos("Hue", window_name)
    process_image()

def update_saturation_tolerance(*args):
    global saturation_tolerance
    saturation_tolerance = cv2.getTrackbarPos("Saturation", window_name)
    process_image()

def update_value_tolerance(*args):
    global value_tolerance
    value_tolerance = cv2.getTrackbarPos("Value", window_name)
    process_image()

# ----------------------------------------------

hue_tolerance = 100
saturation_tolerance = 100
value_tolerance = 100

window_name = "TEST"
cv2.namedWindow(window_name)

cv2.createTrackbar("Hue", window_name, hue_tolerance, 180, update_hue_tolerance)
cv2.createTrackbar("Saturation", window_name, saturation_tolerance, 255, update_saturation_tolerance)
cv2.createTrackbar("Value", window_name, value_tolerance, 255, update_value_tolerance)

process_image()

cv2.waitKey(0)
cv2.destroyAllWindows()

# ----------------------------------------------

### Canny testing

In [3]:
# ----------------------------------------------

with open(input, 'r') as json_file:
    data = json.load(json_file)

image_paths = data['image_files']
image_index = 23

original_image = cv2.imread(os.path.join(images_dir, image_paths[image_index]))
original_image = cv2.resize(original_image, (0, 0), fx = 0.15, fy = 0.15)
image = original_image.copy()

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

complexity, _ = get_image_complexity(gray)

# ----------------------------------------------

def process_image():
    complexity, _ = get_image_complexity(gray)
    blur_kernel = (5, 5)
    if complexity == 2:
        blur_kernel = (5, 5)
    elif complexity == 3:
        blur_kernel = (7, 7)

    canny_mask = canny_edges(gray, blur_kernel, low_threshold, high_threshold)
    canny = cv2.bitwise_and(image, image, mask=canny_mask)
    contours, _ = cv2.findContours(canny_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = filter_contours(contours, original_image)
    image_copy = image.copy()
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(image_copy, (x, y), (x + w, y + h), (255, 255, 0), 2)
    combined_image = np.hstack((canny, image_copy))
    cv2.imshow(window_name, combined_image)

def update_low_threshold(*args):
    global low_threshold
    low_threshold = cv2.getTrackbarPos("Low", window_name)
    process_image()

def update_high_threshold(*args):
    global high_threshold
    high_threshold = cv2.getTrackbarPos("High", window_name)
    process_image()

# ----------------------------------------------

low_threshold = 100
high_threshold = 200

window_name = "TEST"
cv2.namedWindow(window_name, cv2.WINDOW_AUTOSIZE)

cv2.createTrackbar("Low", window_name, low_threshold, 255, update_low_threshold)
cv2.createTrackbar("High", window_name, high_threshold, 255, update_high_threshold)

process_image()

cv2.waitKey(0)
cv2.destroyAllWindows()

# ----------------------------------------------